In [2]:
import os
import logging
import sys
import nest_asyncio
import json

In [3]:
from dotenv import load_dotenv
from llama_index.llms.openai import OpenAI
from llama_index.core import (
    Settings,
    Document, get_response_synthesizer,
    KnowledgeGraphIndex, PropertyGraphIndex,
    StorageContext
)
from llama_index.core.node_parser import SentenceSplitter
from llama_index.graph_stores.neo4j import Neo4jGraphStore

In [4]:
cd ../

/home/tsunn/Workspace/iai-lab/sosci/codes/Graph-RAG


In [5]:
logging.basicConfig(stream=sys.stdout, level=logging.INFO)
logging.getLogger("httpx").setLevel(logging.WARNING)
logging.getLogger("llama_index").setLevel(logging.INFO)
nest_asyncio.apply()

In [14]:
from llm.TogetherLLM import TogetherLLM
llm = TogetherLLM()
Settings.llm = llm
Settings.chunk_size = 512

Custom LLM initialized with model: meta-llama/Meta-Llama-3.1-8B-Instruct-Turbo-128K


In [15]:
graph_store = Neo4jGraphStore(
    username=os.environ["NEO4J_USERNAME"],
    password=os.environ["NEO4J_PASSWORD"],
    url=os.environ["NEO4J_URI"]
)

INFO:neo4j.notifications:Received notification from DBMS server: {severity: INFORMATION} {code: Neo.ClientNotification.Schema.IndexOrConstraintAlreadyExists} {category: SCHEMA} {title: `CREATE CONSTRAINT IF NOT EXISTS FOR (e:Entity) REQUIRE (e.id) IS UNIQUE` has no effect.} {description: `CONSTRAINT constraint_1ed05907 FOR (e:Entity) REQUIRE (e.id) IS UNIQUE` already exists.} {position: None} for query: '\n                CREATE CONSTRAINT IF NOT EXISTS FOR (n:Entity) REQUIRE n.id IS UNIQUE;\n                '


In [ ]:
# graph_store.query(
# """
# MATCH (n) DETACH DELETE n
# """
# )

In [16]:
storage_context = StorageContext.from_defaults(graph_store=graph_store)

In [4]:
data = []
with open('data/annotations.jsonl', 'r') as f:
    for line in f:
        line = json.loads(line)
        for doc in line['documents']:
            data.append(doc["text"])

In [5]:
len(data)

74669

In [ ]:
data = [Document(text=doc) for doc in data]
len(data)

In [13]:
splitter = SentenceSplitter(separator=".")

In [ ]:
chunks = splitter.get_nodes_from_documents(data, show_progress=True)
len(chunks)

In [13]:
index = PropertyGraphIndex(nodes = [],# only reload the graph
                            storage_context=storage_context, 
                            include_embeddings=False,
                            show_progress=True)

ValueError: 
******
Could not load OpenAI embedding model. If you intended to use OpenAI, please check your OPENAI_API_KEY.
Original error:
No API key found for OpenAI.
Please set either the OPENAI_API_KEY environment variable or openai.api_key prior to initialization.
API keys can be found or created at https://platform.openai.com/account/api-keys

Consider using embed_model='local'.
Visit our documentation for more embedding options: https://docs.llamaindex.ai/en/stable/module_guides/models/embeddings.html#modules
******

In [9]:
query_engine = index.as_query_engine(
    include_text=False, response_mode="tree_summarize"
)

NameError: name 'index' is not defined

In [10]:
response = query_engine.query("Tell me more about Interleaf")

NameError: name 'query_engine' is not defined

In [ ]:
response

In [15]:
query_engine = index.as_query_engine(
    include_text=True,
    response_mode="tree_summarize",
    embedding_mode="hybrid",
    similarity_top_k=5
)

In [ ]:
response = query_engine.query(
    "Tell me more about what the author worked on at Interleaf"
)

In [ ]:
response